<a href="https://colab.research.google.com/github/Dobenkss/AssociationRuleMining/blob/main/association-rule-mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_excel('/content/data_apriori.xlsx')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   InvoiceNo            541909 non-null  object        
 1   InvoiceDate          541909 non-null  datetime64[ns]
 2   BRANCH_SPLR          541909 non-null  int64         
 3   BRANCHNAME_SPLR      541909 non-null  object        
 4   warehouseProductsID  541909 non-null  object        
 5   BARCODEID            541909 non-null  int64         
 6   StockCode            541909 non-null  object        
 7   PRODUCT              541909 non-null  object        
 8   PRODUCT_CATEGORY     541909 non-null  object        
 9   Quantity             541909 non-null  int64         
 10  UnitPrice            541909 non-null  float64       
 11  UnitPriceRupiah      541909 non-null  float64       
 12  oldCUSTID            541909 non-null  object        
 13  CustomerID    

In [ ]:
df.head()

,InvoiceNo,InvoiceDate,BRANCH_SPLR,BRANCHNAME_SPLR,warehouseProductsID,BARCODEID,StockCode,PRODUCT,PRODUCT_CATEGORY,Quantity,...,CustomerID,CUSTNAME,ADDRESS,KOTA,PROVINSI,NEGARA,CHANNELID_SPLR,CHANNELNAME_SPLR,SUBDISTID,SUBDIST_NAME
0,536365,2020-12-01 08:26:00,13,SAMARINDA,10001,8992753282401,85123A,123 BENDERA COKLAT 300G,SUSU,6,...,17850.0,DUA PUTRI SLAMET RIYADI,JL. SLAMET RIYADI,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
1,536365,2020-12-01 08:26:00,13,SAMARINDA,20020,8999909192034,71053,2.3.4 FILTER,ROKOK,6,...,17850.0,SANURI,JL. M. SAID,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
2,536365,2020-12-01 08:26:00,19,YOGYAKARTA,20021,8999909007147,84406B,234 KERETEK,ROKOK,8,...,17850.0,EMI MBAK,PS. TALUN NO.63,MAGELANG,JAWA TENGAH,INDONESIA,32,Toko Kelontong,190105,PT. KTRI DISTRIBUSI
3,536365,2020-12-01 08:26:00,13,SAMARINDA,20018,8999909172234,84029G,234 KERETEK 12,ROKOK,6,...,17850.0,RISKA CELL ADAM MALIK,JL. ADAM MALIK,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
4,536365,2020-12-01 08:26:00,19,YOGYAKARTA,20019,8999909005860,84029E,234 KERETEK 16,ROKOK,6,...,17850.0,ABADI MART,JL.GROWONG - PUCUNG REJO MUNTILAN.,MAGELANG,JAWA TENGAH,INDONESIA,42,Mini Market,190105,PT. KTRI DISTRIBUSI


In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
InvoiceDate,0
BRANCH_SPLR,0
BRANCHNAME_SPLR,0
warehouseProductsID,0
BARCODEID,0
StockCode,0
PRODUCT,0
PRODUCT_CATEGORY,0
Quantity,0


In [ ]:
#Cleansing
#ubah tipe data invoiceno menjadi str
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
# Dropping all transactions which cancel
df = df[~df['InvoiceNo'].str.contains('C')]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532621 entries, 0 to 541908
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   InvoiceNo            532621 non-null  object        
 1   InvoiceDate          532621 non-null  datetime64[ns]
 2   BRANCH_SPLR          532621 non-null  int64         
 3   BRANCHNAME_SPLR      532621 non-null  object        
 4   warehouseProductsID  532621 non-null  object        
 5   BARCODEID            532621 non-null  int64         
 6   StockCode            532621 non-null  object        
 7   PRODUCT              532621 non-null  object        
 8   PRODUCT_CATEGORY     532621 non-null  object        
 9   Quantity             532621 non-null  int64         
 10  UnitPrice            532621 non-null  float64       
 11  UnitPriceRupiah      532621 non-null  float64       
 12  oldCUSTID            532621 non-null  object        
 13  CustomerID         

In [ ]:
#cleanup space in product description
df['PRODUCT']=df['PRODUCT'].str.strip()

In [ ]:
#cleanup space in product_category
df['PRODUCT_CATEGORY']=df['PRODUCT_CATEGORY'].str.strip()

In [ ]:
# data transformation into tabular

# Transactions done in Jaw tengah
basket = (df[df['PROVINSI'] =="JAWA TENGAH"]
          .groupby(['InvoiceNo', 'PRODUCT_CATEGORY'])['Quantity']
          .count().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [ ]:
basket.head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
basket.shape

(8121, 52)

In [ ]:
#encoding
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

In [ ]:
# Encoding the datasets
basket_encoded = basket.map(hot_encode)
basket = basket_encoded

In [ ]:
basket.head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536373,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
!pip install apyori
from mlxtend.frequent_patterns import apriori, association_rules

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=42ba04f1309253842952f703ec69d151f322f6de3a419be5f943e195fd6df8e2
  Stored in directory: /root/.cache/pip/wheels/7f/49/e3/42c73b19a264de37129fadaa0c52f26cf50e87de08fb9804af
Successfully built apyori


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Building the model
frequent_items = apriori(basket, min_support = 0.1, use_colnames = True)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
frequent_items

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets
0,0.326438,(ALAT RUMAH TANGGA)
1,0.348726,(BISKUIT)
2,0.172516,(BUMBU)
3,0.166359,(COKELAT)
4,0.273488,(DETERGEN)
...,...,...
250,0.102204,"(SNACK, SABUN & SAMPHOO, KOSMETIK, MINUMAN)"
251,0.117473,"(SABUN & SAMPHOO, SUSU, KOSMETIK, MINUMAN)"
252,0.102081,"(SABUN & SAMPHOO, PARFUM, KOSMETIK, OBATAN)"
253,0.106760,"(SABUN & SAMPHOO, PARFUM, SUSU, KOSMETIK)"


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
x = apriori(basket, min_support = 0.1, use_colnames = True)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# Collecting the inferred rules in a dataframe
rules = association_rules(x, metric ="lift", min_threshold = 1)
rules.head()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(BISKUIT),(ALAT RUMAH TANGGA),0.348726,0.326438,0.177811,0.509887,1.561974,1.0,0.063973,1.374300,0.552431,0.357514,0.272357,0.527294
1,(ALAT RUMAH TANGGA),(BISKUIT),0.326438,0.348726,0.177811,0.544700,1.561974,1.0,0.063973,1.430430,0.534152,0.357514,0.300909,0.527294
2,(ALAT RUMAH TANGGA),(DETERGEN),0.326438,0.273488,0.140377,0.430026,1.572375,1.0,0.051100,1.274641,0.540439,0.305466,0.215465,0.471654
3,(DETERGEN),(ALAT RUMAH TANGGA),0.273488,0.326438,0.140377,0.513282,1.572375,1.0,0.051100,1.383887,0.501051,0.305466,0.277398,0.471654
4,(ALAT RUMAH TANGGA),(KOSMETIK),0.326438,0.412757,0.181997,0.557525,1.350735,1.0,0.047258,1.327179,0.385506,0.326630,0.246522,0.499228


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
rules

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(BISKUIT),(ALAT RUMAH TANGGA),0.348726,0.326438,0.177811,0.509887,1.561974,1.0,0.063973,1.374300,0.552431,0.357514,0.272357,0.527294
1,(ALAT RUMAH TANGGA),(BISKUIT),0.326438,0.348726,0.177811,0.544700,1.561974,1.0,0.063973,1.430430,0.534152,0.357514,0.300909,0.527294
2,(ALAT RUMAH TANGGA),(DETERGEN),0.326438,0.273488,0.140377,0.430026,1.572375,1.0,0.051100,1.274641,0.540439,0.305466,0.215465,0.471654
3,(DETERGEN),(ALAT RUMAH TANGGA),0.273488,0.326438,0.140377,0.513282,1.572375,1.0,0.051100,1.383887,0.501051,0.305466,0.277398,0.471654
4,(ALAT RUMAH TANGGA),(KOSMETIK),0.326438,0.412757,0.181997,0.557525,1.350735,1.0,0.047258,1.327179,0.385506,0.326630,0.246522,0.499228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,"(PARFUM, MINUMAN)","(SABUN & SAMPHOO, SUSU)",0.195173,0.174732,0.102327,0.524290,3.000536,1.0,0.068224,1.734814,0.828409,0.382421,0.423569,0.554957
1276,(SABUN & SAMPHOO),"(SUSU, PARFUM, MINUMAN)",0.364364,0.125600,0.102327,0.280838,2.235967,1.0,0.056563,1.215859,0.869627,0.263977,0.177536,0.547772
1277,(SUSU),"(SABUN & SAMPHOO, PARFUM, MINUMAN)",0.299594,0.140500,0.102327,0.341554,2.430988,1.0,0.060234,1.305346,0.840433,0.302953,0.233919,0.534931
1278,(PARFUM),"(SABUN & SAMPHOO, SUSU, MINUMAN)",0.326191,0.136190,0.102327,0.313703,2.303422,1.0,0.057903,1.258654,0.839798,0.284200,0.205500,0.532530


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
result=rules[(rules['lift']>=1)&(rules['confidence']>=0.8)]
result= result.sort_values(by='confidence', ascending=False)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
result.head()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1239,"(SABUN & SAMPHOO, PARFUM, OBATAN)",(KOSMETIK),0.115503,0.412757,0.102081,0.883795,2.141200,1.0,0.054406,5.053522,0.602571,0.239526,0.802118,0.565555
1252,"(SABUN & SAMPHOO, SUSU, PARFUM)",(KOSMETIK),0.121290,0.412757,0.106760,0.880203,2.132497,1.0,0.056697,4.901986,0.604370,0.249856,0.796001,0.569427
1085,"(SABUN & SAMPHOO, OBATAN, BISKUIT)",(MINUMAN),0.117596,0.383327,0.102697,0.873298,2.278206,1.0,0.057619,4.867128,0.635829,0.257885,0.794540,0.570604
1183,"(SABUN & SAMPHOO, PARFUM, MINUMAN)",(KOSMETIK),0.140500,0.412757,0.122522,0.872042,2.112725,1.0,0.064530,4.589344,0.612772,0.284448,0.782104,0.584440
1056,"(BISKUIT, SABUN & SAMPHOO, PARFUM)",(KOSMETIK),0.127694,0.412757,0.111316,0.871745,2.112006,1.0,0.058610,4.578729,0.603591,0.259397,0.781599,0.570718


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
result

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1239,"(SABUN & SAMPHOO, PARFUM, OBATAN)",(KOSMETIK),0.115503,0.412757,0.102081,0.883795,2.141200,1.0,0.054406,5.053522,0.602571,0.239526,0.802118,0.565555
1252,"(SABUN & SAMPHOO, SUSU, PARFUM)",(KOSMETIK),0.121290,0.412757,0.106760,0.880203,2.132497,1.0,0.056697,4.901986,0.604370,0.249856,0.796001,0.569427
1085,"(SABUN & SAMPHOO, OBATAN, BISKUIT)",(MINUMAN),0.117596,0.383327,0.102697,0.873298,2.278206,1.0,0.057619,4.867128,0.635829,0.257885,0.794540,0.570604
1183,"(SABUN & SAMPHOO, PARFUM, MINUMAN)",(KOSMETIK),0.140500,0.412757,0.122522,0.872042,2.112725,1.0,0.064530,4.589344,0.612772,0.284448,0.782104,0.584440
1056,"(BISKUIT, SABUN & SAMPHOO, PARFUM)",(KOSMETIK),0.127694,0.412757,0.111316,0.871745,2.112006,1.0,0.058610,4.578729,0.603591,0.259397,0.781599,0.570718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,"(SABUN & SAMPHOO, OBATAN)",(MINUMAN),0.166605,0.383327,0.133481,0.801183,2.090075,1.0,0.069617,3.101704,0.625812,0.320520,0.677597,0.574700
958,"(SABUN & SAMPHOO, ALAT RUMAH TANGGA, KOSMETIK)",(MINUMAN),0.133727,0.383327,0.107007,0.800184,2.087470,1.0,0.055745,3.086206,0.601371,0.260961,0.675978,0.539668
961,"(ALAT RUMAH TANGGA, KOSMETIK, MINUMAN)",(SABUN & SAMPHOO),0.133727,0.364364,0.107007,0.800184,2.196112,1.0,0.058281,3.181109,0.628728,0.273615,0.685644,0.546932
714,"(PARFUM, PERMEN)",(KOSMETIK),0.141608,0.412757,0.113287,0.800000,1.938186,1.0,0.054837,2.936215,0.563908,0.256840,0.659425,0.537232


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
